In [125]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

df = pd.concat([train_df,test_df],axis=0,sort=False)
categ_df = df.select_dtypes(include=['object'])
numer_df =df.select_dtypes(exclude=['object'])

In [126]:
#Features with too many NaN values, dropping them
#Also Id which has no correlation to houses, just a sample marker
df = df.drop(["Id","Alley","PoolQC","Fence","MiscFeature"],axis=1)

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 76 columns):
MSSubClass       2919 non-null int64
MSZoning         2915 non-null object
LotFrontage      2433 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2917 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2918 non-null object
Exterior2nd      2918 non-null object
MasVnrType       2895 no

In [131]:
#Show features with at least some NaN values
null_counts = df.isnull().sum()
null_cols = []
print("Number of null values in each column:\n")
for i in range(len(null_counts)):
    if null_counts[i] != 0:
        print(null_counts.index[i], null_counts[i])
        null_cols.append(null_counts.index[i])

Number of null values in each column:

MSZoning 4
LotFrontage 486
Utilities 2
Exterior1st 1
Exterior2nd 1
MasVnrType 24
MasVnrArea 23
BsmtFinSF1 1
BsmtFinSF2 1
BsmtUnfSF 1
TotalBsmtSF 1
Electrical 1
BsmtFullBath 2
BsmtHalfBath 2
KitchenQual 1
Functional 2
GarageYrBlt 159
GarageCars 1
GarageArea 1
SaleType 1
SalePrice 1459


In [132]:
#Columns that have the data description with an explanation for NaN values
##Basically indicates a lack of the feature is specified
valid_none_cols = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','GarageType',
                'GarageFinish','GarageQual','FireplaceQu','GarageCond']

##Filling in with value 'None' to indicate for machine learning later
df[valid_none_cols]= df[valid_none_cols].fillna('None')

In [133]:
#Organizing the other features with NaN values into numerical and categorical features
#"Invalid" simply means the data description didn't specify NaN values

invalid_none_cols = np.array(null_cols)
invalid_none_cols = invalid_none_cols[[i not in valid_none_cols for i in invalid_none_cols]]

invalid_none_categ = invalid_none_cols[[i in categ_df for i in invalid_none_cols]]
invalid_none_numer = invalid_none_cols[[i in numer_df for i in invalid_none_cols]]

#Deletes SalePrice which shows up due to being empty for the test set
#Definitely not something we want to fill in with values yet
invalid_none_numer = np.delete(invalid_none_numer,-1)

print(invalid_none_categ)
print(invalid_none_numer)
#fill missing values for each column (using its own most frequent value)
#object_columns_df[columns_with_lowNA] = object_columns_df[columns_with_lowNA].fillna(object_columns_df.mode().iloc[0])

['MSZoning' 'Utilities' 'Exterior1st' 'Exterior2nd' 'MasVnrType'
 'Electrical' 'KitchenQual' 'Functional' 'SaleType']
['LotFrontage' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' 'BsmtFullBath' 'BsmtHalfBath' 'GarageYrBlt' 'GarageCars'
 'GarageArea']


In [134]:
#Now we can adjust the NaN values for the rest according to type
#Numerical features can be filled based on their most median value
#Categorical features can be filled by their most common (mode) value
#Not perfect, but likely a good enough approximation to the missing values in the data

#ADJUSTING THE CATEGORICAL VALUES IN LINE WITH THE MODES OF THEIR DATA
for i in invalid_none_categ:
    mode = df[i].mode().to_numpy()[0]
    df[i] = df[i].fillna(value = mode)
#print(df.iloc[1379,40]) #This value is NaN until changed to SBrkr

In [136]:
##ADJUSTING NUMERICAL FEATURES
#For the most part these values can be assumed to indicate 0 when not filled with a number
##LotFrontage has many missing values, so all 0 wouldn't be great
#GarageYrBlt makes no sense with a value of 0. We'll change these two individually

#Adjusting LotFrontage by assuming the fill takes on the median value
#Perhaps this can be improved with comparisons to the LotArea feature?
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())


#Adjusting GarageYrBlt based on the year the house was built

#plt.plot(train_df_NA["GarageYrBlt"],train_df_NA["YearBuilt"],"b.")
#plt.xticks(np.arange(min(train_df_NA["GarageYrBlt"]),max(train_df_NA["GarageYrBlt"]),20.0))
#plt.xlabel("Year Garage Built")
#plt.ylabel("Year Built")

#seems like the garage is built around the same year as the house or a bit later
#Again use the median value to fill in
df["GarageYrBlt"] = df["GarageYrBlt"].fillna(df['YearBuilt']-(df['GarageYrBlt']-df['YearBuilt']).median())

#Fill the rest of the null values to 0
for i in invalid_none_numer:
    df[i] = df[i].fillna(0)

In [137]:
#Show features with at least some NaN values
null_counts = df.isnull().sum()
null_cols = []
print("Number of null values in each column:\n")
for i in range(len(null_counts)):
    if null_counts[i] != 0:
        print(null_counts.index[i], null_counts[i])
        null_cols.append(null_counts.index[i])

##As desired the null values are only the SalePrice from the test set now

Number of null values in each column:

SalePrice 1459


In [140]:
#Split df back into test and train set
train_df_clean = df[0:1460]
test_df_clean = df[1460:]

### Notes
- Some features, like 'pool', seem to mostly have all the same values. Perhaps worth dropping these as well?
- Maybe worth checking the logic of some of the data, like year features or square footage
    - Can't have a house built after it's sold or a bathroom larger than the house!